In [1]:
import pandas as pd
from functools import reduce

In [2]:
pop_col_list = ['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State', 'POP2013', 'POP13_SQMI']
age_col_list = ['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State', 'MED_AGE']
household_col_list = ['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State', 'HOUSEHOLDS','AVE_HH_SZ']
disease_col_list = ['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State', 'Prevalence of obesity', 'Diabetes']
all_col_list = ['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State', 'POP2013', 'POP13_SQMI', 'MED_AGE', 'HOUSEHOLDS','AVE_HH_SZ', 'Prevalence of obesity', 'Diabetes']
all_col_dict = {'POP2013':'pop_2013','POP13_SQMI':'pop_2013_sqmi','MED_AGE':'median_age','HOUSEHOLDS':'num_households', 'AVE_HH_SZ':'avg_household_size', 'Prevalence of obesity':'obesity_prevalence', 'Diabetes':'diabetes', 'num pharmacies':'num_pharmacies'}

In [3]:
pharmacy_data = pd.read_excel('Pharmacy-County-2sheets.xlsx', sheet_name="With County Info").iloc[:,:6]#['State', 'Pharmacy Name', 'Street Address', 'City', 'Zip county']
pharmacy_data.head()

,State,Pharmacy Name,Street Address,City,Zip,county
0,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,Credena Health Pharmacy Anchorage,3300 PROVIDENCE DR STE 101,ANCHORAGE,99508.0,Anchorage
2,Alaska,FRED MEYER #00011,1000 E Northern Lights Blvd,ANCHORAGE,99508.0,Anchorage
3,Alaska,RELIANCE MEDSETS,"1035 W. FIREWEED LN., STE 100",ANCHORAGE,99503.0,Anchorage
4,Alaska,CARRS PHARMACY #1807,11431 Business Blvd,EAGLE RIVER,99577.0,Anchorage


In [4]:
pharmacy_data = pharmacy_data.dropna(how='all')
pharmacy_data.head()

,State,Pharmacy Name,Street Address,City,Zip,county
1,Alaska,Credena Health Pharmacy Anchorage,3300 PROVIDENCE DR STE 101,ANCHORAGE,99508.0,Anchorage
2,Alaska,FRED MEYER #00011,1000 E Northern Lights Blvd,ANCHORAGE,99508.0,Anchorage
3,Alaska,RELIANCE MEDSETS,"1035 W. FIREWEED LN., STE 100",ANCHORAGE,99503.0,Anchorage
4,Alaska,CARRS PHARMACY #1807,11431 Business Blvd,EAGLE RIVER,99577.0,Anchorage
5,Alaska,FAMILY PHARMACY,11432 BUSINESS BLVD SUITE 10,EAGLE RIVER,99577.0,Anchorage


In [5]:
county_pharmacy_counts = pd.DataFrame(pharmacy_data.groupby(['State','county']).agg({'Pharmacy Name':['count']})).reset_index()
county_pharmacy_counts.columns = county_pharmacy_counts.columns.droplevel(1)
county_pharmacy_counts.rename(columns={'Pharmacy Name':'num pharmacies'},inplace=True)
county_pharmacy_counts.to_excel('County Counts.xlsx', index=False)
county_pharmacy_counts.head()

,State,county,num pharmacies
0,Alaska,Anchorage,51
1,Alaska,Dillingham,1
2,Alaska,Fairbanks North Star,13
3,Alaska,Haines,1
4,Alaska,Juneau,7


In [8]:
pop_data = pd.read_excel("big_data_pharmacy_county/Demography_USA.xlsx", sheet_name = 'Population')[pop_col_list]
pop_data.head()

,OBJECTID,NAME,County Name,STATE_NAME,State,POP2013,POP13_SQMI
0,651,Randolph,Randolph,Alabama,AL,22974,39.3
1,791,Henry,Henry,Alabama,AL,17375,30.6
2,808,Lamar,Lamar,Alabama,AL,14530,24.0
3,846,Dale,Dale,Alabama,AL,50154,89.1
4,912,Coffee,Coffee,Alabama,AL,51244,75.3


In [9]:
age_data = pd.read_excel("Demography_USA.xlsx", sheet_name = 'Ages')[age_col_list]
age_data.head()

,OBJECTID,NAME,County Name,STATE_NAME,State,MED_AGE
0,651,Randolph,Randolph,Alabama,AL,41.1
1,791,Henry,Henry,Alabama,AL,42.8
2,808,Lamar,Lamar,Alabama,AL,43.5
3,846,Dale,Dale,Alabama,AL,36.1
4,912,Coffee,Coffee,Alabama,AL,37.6


In [10]:
household_data = pd.read_excel("Demography_USA.xlsx", sheet_name = 'Households')[household_col_list]
household_data.head()


,OBJECTID,NAME,County Name,STATE_NAME,State,HOUSEHOLDS,AVE_HH_SZ
0,651,Randolph,Randolph,Alabama,AL,9164,2.46
1,791,Henry,Henry,Alabama,AL,6994,2.45
2,808,Lamar,Lamar,Alabama,AL,6103,2.35
3,846,Dale,Dale,Alabama,AL,20065,2.46
4,912,Coffee,Coffee,Alabama,AL,19849,2.49


In [11]:
household_data.shape

(3219, 7)

In [12]:
diesease_data = pd.read_excel("Demography_USA.xlsx", sheet_name = 'Diseases')[disease_col_list]
diesease_data.head()

,OBJECTID,NAME,County Name,STATE_NAME,State,Prevalence of obesity,Diabetes
0,651,Randolph,Randolph,Alabama,AL,6364,2813
1,791,Henry,Henry,Alabama,AL,4779,2188
2,808,Lamar,Lamar,Alabama,AL,3803,2003
3,846,Dale,Dale,Alabama,AL,13652,5806
4,912,Coffee,Coffee,Alabama,AL,12938,6099


In [13]:
dfs = [pop_data, age_data, household_data, diesease_data]
df_final = reduce(lambda left,right: pd.merge(left,right,on=['OBJECTID', 'NAME', 'County Name', 'STATE_NAME', 'State']), dfs)
df_final = df_final[all_col_list]
df_final.head()

,OBJECTID,NAME,County Name,STATE_NAME,State,POP2013,POP13_SQMI,MED_AGE,HOUSEHOLDS,AVE_HH_SZ,Prevalence of obesity,Diabetes
0,651,Randolph,Randolph,Alabama,AL,22974,39.3,41.1,9164,2.46,6364,2813
1,791,Henry,Henry,Alabama,AL,17375,30.6,42.8,6994,2.45,4779,2188
2,808,Lamar,Lamar,Alabama,AL,14530,24.0,43.5,6103,2.35,3803,2003
3,846,Dale,Dale,Alabama,AL,50154,89.1,36.1,20065,2.46,13652,5806
4,912,Coffee,Coffee,Alabama,AL,51244,75.3,37.6,19849,2.49,12938,6099


In [14]:
combine_dfs = [df_final, county_pharmacy_counts]
counts_final_df = pd.merge(df_final,county_pharmacy_counts,left_on=['STATE_NAME','County Name'],right_on=['State','county'])
counts_final_df.head()

,OBJECTID,NAME,County Name,STATE_NAME,State_x,POP2013,POP13_SQMI,MED_AGE,HOUSEHOLDS,AVE_HH_SZ,Prevalence of obesity,Diabetes,State_y,county,num pharmacies
0,2958,Anchorage,Anchorage,Alaska,AK,300046,173.1,33.0,107332,2.64,69213,16977,Alaska,Anchorage,51
1,3179,Haines,Haines,Alaska,AK,2621,1.1,46.9,1149,2.18,552,152,Alaska,Haines,1
2,3181,Juneau,Juneau,Alaska,AK,32723,11.9,38.0,12187,2.49,6702,1589,Alaska,Juneau,7
3,3184,Fairbanks North Star,Fairbanks North Star,Alaska,AK,103018,13.9,31.2,36441,2.56,22067,4594,Alaska,Fairbanks North Star,13
4,3194,Matanuska-Susitna,Matanuska-Susitna,Alaska,AK,97147,3.9,34.8,31824,2.75,25287,6372,Alaska,Matanuska-Susitna,9


In [16]:
pharmacy_data_demography = counts_final_df.drop(['State_x', 'NAME', 'STATE_NAME', 'State_y', 'county', 'County Name'],axis=1).rename(columns=all_col_dict)
pharmacy_data_demography.head()

,OBJECTID,pop_2013,pop_2013_sqmi,median_age,num_households,avg_household_size,obesity_prevalence,diabetes,num_pharmacies
0,2958,300046,173.1,33.0,107332,2.64,69213,16977,51
1,3179,2621,1.1,46.9,1149,2.18,552,152,1
2,3181,32723,11.9,38.0,12187,2.49,6702,1589,7
3,3184,103018,13.9,31.2,36441,2.56,22067,4594,13
4,3194,97147,3.9,34.8,31824,2.75,25287,6372,9


In [17]:
data_for_prediction = pharmacy_data_demography
data_for_prediction.head()

,OBJECTID,pop_2013,pop_2013_sqmi,median_age,num_households,avg_household_size,obesity_prevalence,diabetes,num_pharmacies
0,2958,300046,173.1,33.0,107332,2.64,69213,16977,51
1,3179,2621,1.1,46.9,1149,2.18,552,152,1
2,3181,32723,11.9,38.0,12187,2.49,6702,1589,7
3,3184,103018,13.9,31.2,36441,2.56,22067,4594,13
4,3194,97147,3.9,34.8,31824,2.75,25287,6372,9


In [18]:
data_for_prediction.to_csv('Complete_data_for_predictor.csv', index=False)